# Engenharia e seleção de features

# 1 - Lendo os dados

In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_halving_search_cv # Necessário para importar
from sklearn.model_selection import HalvingRandomSearchCV, KFold
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import root_mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import uniform, randint # Para a distribuição de parâmetros
from sklearn.model_selection import GroupKFold


# Numero de neuronios de entrada na rede neural
neuronios_entrada = 300

2025-10-15 13:50:20.089230: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 13:50:20.089439: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 13:50:20.129754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-15 13:50:21.024807: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
df = pd.read_csv('../data/02_intermediate/training_b_df.csv')

In [3]:
X_train = df.drop(columns=['r'])
y_train = df['r']

In [4]:
X_train

,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,l_11,l_12,l_13,l_14,l_15,l_16,l_17,l_18,l_19,l_20,l_21,l_22,l_23,l_24,l_25,l_26,l_27,l_28,l_29,l_30,l_31,l_32,l_33,l_34,l_35,l_36,l_37,l_38,l_39,l_40,l_41,...,l_472,l_473,l_474,l_475,l_476,l_477,l_478,l_479,l_480,l_481,l_482,l_483,l_484,l_485,l_486,l_487,l_488,l_489,l_490,l_491,l_492,l_493,l_494,l_495,l_496,l_497,l_498,l_499,l_500,l_501,l_502,l_503,l_504,l_505,l_506,l_507,l_508,l_509,l_510,l_511
0,0.000172,0.000066,0.000078,0.000069,0.000063,0.000021,0.000053,0.000031,0.000043,0.000060,0.000067,0.000051,0.000061,0.000070,0.000127,0.000098,0.000125,0.000164,0.000139,0.000263,0.000228,0.000213,0.000207,0.000210,0.000266,0.000202,0.000262,0.000251,0.000258,0.000288,0.000262,0.000327,0.000409,0.000339,0.000496,0.000481,0.000514,0.000467,0.000551,0.000417,...,3.933667e-09,3.681910e-09,3.233619e-09,3.616531e-09,3.082747e-09,3.058790e-09,2.775068e-09,2.571999e-09,2.576051e-09,2.225300e-09,2.154922e-09,2.148977e-09,1.977523e-09,1.790030e-09,1.772219e-09,1.670784e-09,1.465850e-09,1.522312e-09,1.346115e-09,1.306253e-09,1.225206e-09,1.140412e-09,1.120225e-09,1.019040e-09,9.889522e-10,9.027699e-10,8.488432e-10,7.974954e-10,7.367031e-10,6.883090e-10,6.490171e-10,6.058483e-10,6.123373e-10,5.492230e-10,4.987756e-10,4.996063e-10,4.873860e-10,4.858634e-10,3.759005e-10,4.054609e-10
1,0.000058,0.000068,0.000118,0.000045,0.000045,0.000043,0.000063,0.000035,0.000042,0.000061,0.000055,0.000061,0.000066,0.000085,0.000089,0.000179,0.000095,0.000101,0.000158,0.000195,0.000144,0.000183,0.000233,0.000232,0.000203,0.000229,0.000235,0.000286,0.000296,0.000312,0.000330,0.000402,0.000307,0.000405,0.000440,0.000466,0.000472,0.000485,0.000346,0.000543,...,3.649750e-09,3.812733e-09,3.306885e-09,3.290945e-09,3.090880e-09,2.836339e-09,2.522960e-09,2.523749e-09,2.332602e-09,2.328704e-09,2.190022e-09,2.025444e-09,1.825829e-09,1.755871e-09,1.777902e-09,1.532477e-09,1.574359e-09,1.302631e-09,1.490625e-09,1.244136e-09,1.272852e-09,1.108928e-09,1.130770e-09,9.608558e-10,1.025681e-09,8.579955e-10,8.648400e-10,7.465398e-10,7.611644e-10,7.196621e-10,6.590990e-10,5.957533e-10,5.382490e-10,5.511306e-10,4.965663e-10,4.848698e-10,4.638441e-10,4.146040e-10,3.850935e-10,3.658829e-10
2,0.000066,0.000086,0.000129,0.000044,0.000056,0.000038,0.000041,0.000059,0.000060,0.000048,0.000065,0.000069,0.000097,0.000092,0.000091,0.000074,0.000104,0.000097,0.000137,0.000122,0.000179,0.000204,0.000269,0.000239,0.000282,0.000341,0.000298,0.000383,0.000364,0.000351,0.000393,0.000350,0.000531,0.000427,0.000574,0.000608,0.000712,0.000481,0.000430,0.000627,...,4.333220e-09,3.658479e-09,3.431150e-09,3.194187e-09,3.002993e-09,2.784651e-09,3.128016e-09,2.254815e-09,2.656183e-09,2.144598e-09,2.200398e-09,1.981116e-09,1.954271e-09,1.786031e-09,1.792107e-09,1.382613e-09,1.674757e-09,1.341786e-09,1.358488e-09,1.171354e-09,1.234611e-09,1.008740e-09,1.064591e-09,8.395177e-10,8.988281e-10,7.834980e-10,7.819893e-10,6.992766e-10,7.224283e-10,5.825830e-10,6.345819e-10,6.044535e-10,5.756162e-10,5.306156e-10,4.744517e-10,4.625054e-10,4.354919e-10,3.960931e-10,3.977156e-10,3.608611e-10
3,0.000140,0.000167,0.000060,0.000070,0.000064,0.000030,0.000018,0.000049,0.000025,0.000042,0.000067,0.000112,0.000072,0.000070,0.000104,0.000083,0.000143,0.000121,0.000108,0.000240,0.000213,0.000305,0.000215,0.000277,0.000304,0.000331,0.000255,0.000291,0.000384,0.000415,0.000451,0.000333,0.000403,0.000432,0.000428,0.000448,0.000395,0.000456,0.000488,0.000715,...,3.646149e-09,3.715941e-09,3.634678e-09,3.654359e-09,3.355548e-09,3.137426e-09,2.743997e-09,2.747997e-09,2.653482e-09,2.321538e-09,2.205917e-09,2.070924e-09,2.046749e-09,1.758527e-09,1.974782e-09,1.487913e-09,1.674413e-09,1.429402e-09,1.409174e-09,1.198962e-09,1.311089e-09,1.105671e-09,1.129333e-09,9.097917e-10,1.032100e-09,7.928750e-10,8.742719e-10,7.423079e-10,7.184108e-10,6.348654e-10,6.901658e-10,6.067260e-10,5.923192e-10,5.206132e-10,5.064276e-10,4.546328e-10,4.771503e-10,4.041781e-10,3.925857e-10,3.493796e-10
4,0.000025,0.000066

In [5]:
y_train

0       0.002872
1       0.002872
2       0.002872
3       0.002872
4       0.002872
          ...   
7995    0.011467
7996    0.011467
7997    0.011467
7998    0.011467
7999    0.011467
Name: r, Length: 8000, dtype: float64

# 2 - Engenharia de features

Ao conversar com a astrofísica Camila Novaes que forneceu os dados simulados estudados aqui, foi sugerida a criação da média ponderada abaixo:

In [ ]:
def cria_feature_media_ponderada(df):
    """
    Cria uma feature fazendo-se a média ponderada dos coeficientes do espectro de potencias da CMB.    
    """
    l_start = 2
    l_end = 511
   
    # Vetor l (o valor do momento de multipolo)
    l_values = np.arange(l_start, l_end + 1)

    # Vetor de pesos: l * (l + 1)
    weights = l_values * (l_values + 1) / 2*np.pi 

    # 4. Denominador da média ponderada (soma dos pesos)
    sum_of_weights = weights.sum()

    # 5. Cálculo do Numerador e da Nova Feature
    # O Pandas alinha as colunas de Cl_data com o vetor 'weights' e multiplica.
    # Depois, .sum(axis=1) soma o resultado dessa multiplicação ao longo das colunas,
    # resultando no numerador para CADA linha (observação).
    numerator = (df * weights).sum(axis=1)

    # Média ponderada
    df['media_ponderada_Cls'] = numerator / sum_of_weights

    return df





In [7]:
X_train = cria_feature_media_ponderada(X_train)

print("Primeiras 5 linhas com a nova feature:")
print(X_train[['l_2', 'l_3', 'l_4', 'l_511', 'media_ponderada_Cls']].head())

Primeiras 5 linhas com a nova feature:
        l_2       l_3       l_4         l_511  media_ponderada_Cls
0  0.000172  0.000066  0.000078  4.054609e-10             0.000100
1  0.000058  0.000068  0.000118  3.658829e-10             0.000100
2  0.000066  0.000086  0.000129  3.608611e-10             0.000098
3  0.000140  0.000167  0.000060  3.493796e-10             0.000100
4  0.000025  0.000066  0.000041  3.678812e-10             0.000099


In [8]:
X_train['media_ponderada_Cls'].isnull().sum()

np.int64(0)

In [9]:
X_train['media_ponderada_Cls'].tail()

7995    0.000096
7996    0.000095
7997    0.000096
7998    0.000096
7999    0.000096
Name: media_ponderada_Cls, dtype: float64

# 3 - Seleção de features

Para a seleção de features utiliza-se o seguinte fluxo:

1 - Utiliza-se o método SelectKBest do scikit-learn para se verificar correlações não-lineares.

2 - Elimina-se as 350 features menos importantes de acordo com o critério acima.

3 - Verifica-se o desempenho da rede neural no conjunto de teste utilizando-se apenas as features selecionadas e compara-se com o desempenho obtido utilizando-se todas as features.

In [10]:
from sklearn.feature_selection import SelectKBest, f_regression

# 1. Instanciar o seletor. K=150 seleciona as 150 melhores features.
k_best = SelectKBest(score_func=f_regression, k=neuronios_entrada)

# 2. Ajustar o seletor aos dados de TREINO (X e y)
k_best.fit(X_train, y_train.to_numpy().ravel())

# 3. Obter os scores e os índices das colunas selecionadas
scores = k_best.scores_
selected_features_indices = k_best.get_support(indices=True)
selected_feature_names = X_train.columns[selected_features_indices]

print("Features Selecionadas:", selected_feature_names)

Features Selecionadas: Index(['l_2', 'l_3', 'l_4', 'l_5', 'l_6', 'l_7', 'l_8', 'l_9', 'l_10', 'l_11',
       ...
       'l_292', 'l_294', 'l_295', 'l_296', 'l_297', 'l_300', 'l_301', 'l_302',
       'l_310', 'media_ponderada_Cls'],
      dtype='object', length=300)


In [11]:
X_train = X_train[selected_feature_names]

In [12]:
X_train

,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,l_11,l_12,l_13,l_14,l_15,l_16,l_17,l_18,l_19,l_20,l_21,l_22,l_23,l_24,l_25,l_26,l_27,l_28,l_29,l_30,l_31,l_32,l_33,l_34,l_35,l_36,l_37,l_38,l_39,l_40,l_41,...,l_262,l_263,l_264,l_265,l_266,l_267,l_268,l_269,l_270,l_271,l_272,l_273,l_274,l_275,l_276,l_277,l_278,l_279,l_280,l_281,l_282,l_283,l_284,l_285,l_286,l_287,l_288,l_289,l_290,l_291,l_292,l_294,l_295,l_296,l_297,l_300,l_301,l_302,l_310,media_ponderada_Cls
0,0.000172,0.000066,0.000078,0.000069,0.000063,0.000021,0.000053,0.000031,0.000043,0.000060,0.000067,0.000051,0.000061,0.000070,0.000127,0.000098,0.000125,0.000164,0.000139,0.000263,0.000228,0.000213,0.000207,0.000210,0.000266,0.000202,0.000262,0.000251,0.000258,0.000288,0.000262,0.000327,0.000409,0.000339,0.000496,0.000481,0.000514,0.000467,0.000551,0.000417,...,0.000121,0.000129,0.000137,0.000116,0.000106,0.000108,0.000108,0.000110,0.000098,0.000088,0.000094,0.000093,0.000089,0.000085,0.000085,0.000076,0.000070,0.000075,0.000072,0.000068,0.000071,0.000063,0.000063,0.000062,0.000058,0.000052,0.000054,0.000049,0.000049,0.000043,0.000041,0.000042,0.000041,0.000036,0.000035,0.000031,0.000031,0.000028,0.000022,0.000100
1,0.000058,0.000068,0.000118,0.000045,0.000045,0.000043,0.000063,0.000035,0.000042,0.000061,0.000055,0.000061,0.000066,0.000085,0.000089,0.000179,0.000095,0.000101,0.000158,0.000195,0.000144,0.000183,0.000233,0.000232,0.000203,0.000229,0.000235,0.000286,0.000296,0.000312,0.000330,0.000402,0.000307,0.000405,0.000440,0.000466,0.000472,0.000485,0.000346,0.000543,...,0.000124,0.000127,0.000120,0.000110,0.000121,0.000101,0.000096,0.000098,0.000100,0.000090,0.000089,0.000090,0.000087,0.000080,0.000084,0.000084,0.000082,0.000072,0.000066,0.000065,0.000063,0.000062,0.000063,0.000058,0.000055,0.000055,0.000053,0.000049,0.000048,0.000047,0.000040,0.000041,0.000041,0.000035,0.000036,0.000033,0.000029,0.000029,0.000022,0.000100
2,0.000066,0.000086,0.000129,0.000044,0.000056,0.000038,0.000041,0.000059,0.000060,0.000048,0.000065,0.000069,0.000097,0.000092,0.000091,0.000074,0.000104,0.000097,0.000137,0.000122,0.000179,0.000204,0.000269,0.000239,0.000282,0.000341,0.000298,0.000383,0.000364,0.000351,0.000393,0.000350,0.000531,0.000427,0.000574,0.000608,0.000712,0.000481,0.000430,0.000627,...,0.000126,0.000127,0.000110,0.000118,0.000102,0.000117,0.000108,0.000105,0.000113,0.000095,0.000093,0.000101,0.000082,0.000073,0.000076,0.000071,0.000071,0.000072,0.000071,0.000064,0.000070,0.000067,0.000058,0.000055,0.000051,0.000050,0.000051,0.000051,0.000049,0.000049,0.000042,0.000040,0.000042,0.000036,0.000034,0.000035,0.000032,0.000029,0.000020,0.000098
3,0.000140,0.000167,0.000060,0.000070,0.000064,0.000030,0.000018,0.000049,0.000025,0.000042,0.000067,0.000112,0.000072,0.000070,0.000104,0.000083,0.000143,0.000121,0.000108,0.000240,0.000213,0.000305,0.000215,0.000277,0.000304,0.000331,0.000255,0.000291,0.000384,0.000415,0.000451,0.000333,0.000403,0.000432,0.000428,0.000448,0.000395,0.000456,0.000488,0.000715,...,0.000133,0.000130,0.000121,0.000125,0.000118,0.000111,0.000113,0.000105,0.000095,0.000091,0.000092,0.000081,0.000086,0.000086,0.000082,0.000073,0.000067,0.000077,0.000070,0.000067,0.000070,0.000060,0.000062,0.000054,0.000057,0.000055,0.000048,0.000049,0.000049,0.000043,0.000042,0.000044,0.000043,0.000034,0.000040,0.000035,0.000031,0.000029,0.000023,0.000100
4,0.000025,0.000066,0.000041,0.000060,0.000052,0.000034,0.000046,0.000031,0.000035,0.000084,0.000039,0.000040,0.000062,0.000079,0.000075,0.000139,0.000116,0.000128,0.000219,0.000093,0.000312,0.000234,0.000318,0.000328,0.000249,0.000356,0.000305,0.000261,0.000274,0.000301,0.000389,0.000431,0.000439,0.000458,0.000360,0.000557,0.000503,0.000473,0.000626,0.000518,...,0.000124,0.000131,0.000126,0.000125,0.000126,0.000108,0.000106,0.000098,0.000093,0.000099,0.000086,0.000084,0.000086,0.000082,0.000075,0.000091,0.000064,0.000079,0.000072,0.000071,0.000062,0.000059,0.000055,0.000058,0.000052,0.000053,0.000049,0.000048,0.000047,0.000044,0.000046,

## 4 - Normalizando os dados

In [13]:
from sklearn.preprocessing import MinMaxScaler

# 1. Escalonamento das Features (X) 
X_scaler = MinMaxScaler()
X_train_scaled = X_scaler.fit_transform(X_train)

# 2. Escalonamento do Target (Y), se for Regressão Contínua 
# Reformatar y_train para que o scaler funcione (de Series para 2D array/DataFrame)
y_train_2d = y_train.values.reshape(-1, 1)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train_2d)


In [14]:
X_train_scaled

array([[0.02784866, 0.00992461, 0.01283325, ..., 0.35285855, 0.49631627,
        0.2532313 ],
       [0.00930768, 0.01028335, 0.01942006, ..., 0.38024816, 0.48990313,
        0.25787837],
       [0.01070404, 0.01302107, 0.0212306 , ..., 0.41922414, 0.39826932,
        0.23394406],
       ...,
       [0.01358998, 0.00411513, 0.0038826 , ..., 0.22447221, 0.5126833 ,
        0.18901289],
       [0.00104116, 0.01890849, 0.00380015, ..., 0.15093673, 0.30976497,
        0.18209283],
       [0.01370877, 0.0070308 , 0.00711748, ..., 0.22279094, 0.26156765,
        0.18226959]], shape=(8000, 300))

In [15]:
X_train_scaled.max()

np.float64(1.0000000000000002)

In [16]:
X_train_scaled.min()

np.float64(0.0)

In [17]:
y_train_scaled.max()

np.float64(1.0)

In [18]:
y_train_scaled.min()

np.float64(0.0)

# 3 - Construindo um modelo de redes neurais

In [19]:
def criar_modelo_regularizado(neurons=64, learning_rate=0.001, dropout_rate=0.3):
    """Cria e compila um modelo de Rede Neural com Dropout."""
    
    optimizer = Adam(learning_rate=learning_rate)
    
    model = Sequential([
        # 1. Camada de Entrada
        Dense(neurons, activation='relu', input_shape=(neuronios_entrada,)),
        
        # 2. DROPOUT (Descarta 30% dos neurônios aleatoriamente)
        Dropout(dropout_rate), 
        
        # 3. Segunda Camada Oculta
        Dense(32, activation='relu'), 
        
        # 4. DROPOUT (Ajuda a evitar que a segunda camada decore)
        Dropout(dropout_rate), 
        
        # 5. Camada de Saída
        Dense(1, activation='linear')
    ])
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

In [20]:
# 1. Distribuição de Parâmetros a serem testados (ranges ao invés de listas fixas)
param_distributions = {
    # Neurônios: número inteiro aleatório entre 32 e 256
    'model__neurons': randint(low=32, high=256), 
    
    # Taxa de Aprendizado: valor contínuo aleatório em escala logarítmica
    # Ex: entre 1e-4 e 1e-2 (0.0001 e 0.01)
    'model__learning_rate': uniform(loc=0.0001, scale=0.0099), 
    
    # Batch Size: valores discretos
    'batch_size': [16, 32, 64],

    # O dropout_rate ajuda a diminuir o overfitting
    'model__dropout_rate': uniform(loc=0.1, scale=0.4), # Testar entre 10% e 50%
    
    # Epochs: valores discretos (o HRS vai descartar os piores cedo)
    'epochs': [5, 10, 20] 
}

# 2. Configurar o KerasRegressor
nn_model = KerasRegressor(model=criar_modelo_regularizado, verbose=0)

## Atenção

- Ao juntar as features com o target, foram colocadas 10 simulações associadas ao mesmo target. 

- Ao usar a validação cruzada é preciso ter certeza de que as linhas associadas a um dado valor do target, caiam tanto na validação quanto no treino, evitando assim o vazamento do target. 

- Para isso utiliza-se o parâmetro `cv=gkf` no HalvingRandomSearch abaixo.

- A variável gkf usa a classe GroupKFold para levar em consideração a observação acima e evitar o vazamento do target.

In [21]:
groups = y_train.to_numpy().ravel() 

In [22]:
groups

array([0.002872, 0.002872, 0.002872, ..., 0.011467, 0.011467, 0.011467],
      shape=(8000,))

Abaixo, verifica-se se realmente não há vazamento de target usando-se o GrupoKFold

In [23]:
# Inicialize o GroupKFold
gkf = GroupKFold(n_splits=5) 

# Itere sobre os splits (apenas a primeira dobra é suficiente para checar)
for fold, (train_index, val_index) in enumerate(gkf.split(X_train, y_train, groups=groups)):
    
    if fold == 0: # Checar apenas o primeiro fold
        
        # 1. Obter os valores de 'r' (originais, não escalados) para Treino e Validação
        r_train_fold = y_train[train_index].to_numpy().ravel()
        r_val_fold = y_train[val_index].to_numpy().ravel()
        
        # 2. Encontrar os valores ÚNICOS de 'r' em cada conjunto
        r_unique_train = set(r_train_fold)
        r_unique_val = set(r_val_fold)
        
        # 3. Encontrar a Interseção (os valores vazados)
        vazamentos = r_unique_train.intersection(r_unique_val)
        
        print(f"--- Checagem do Fold {fold + 1} ---")
        print(f"Total de valores únicos de 'r' no Treino: {len(r_unique_train)}")
        print(f"Total de valores únicos de 'r' na Validação: {len(r_unique_val)}")
        print(f"Valores de 'r' vazando (Interseção): {len(vazamentos)}")
        
        if len(vazamentos) == 0:
            print("✅ GroupKFold está funcionando corretamente: Nenhuma intersecção de valores de 'r'.")
        else:
            print(f"❌ ERRO GRAVE: {len(vazamentos)} valores de 'r' estão vazando! O GroupKFold falhou na divisão dos grupos.")
            print(f"Valores vazados (Primeiros 5): {list(vazamentos)[:5]}")
            
        break # Parar após o primeiro fold

--- Checagem do Fold 1 ---
Total de valores únicos de 'r' no Treino: 640
Total de valores únicos de 'r' na Validação: 160
Valores de 'r' vazando (Interseção): 0
✅ GroupKFold está funcionando corretamente: Nenhuma intersecção de valores de 'r'.


In [ ]:
# factor=2: Descarta metade dos candidatos a cada iteração.
# candidates: O número total de combinações que serão testadas na 1ª rodada (a maior).
hrs = HalvingRandomSearchCV(
    estimator=nn_model, 
    param_distributions=param_distributions, 
    factor=2, 
    n_candidates=50,
    scoring='r2', 
    random_state=42,
    cv=gkf, 
    verbose=2,
    n_jobs=-1 
)

print("Iniciando Halving Random Search (Testando as melhores combinações eficientemente)...")

# 4. Executar a busca
# O HRS executa a busca e o retreinamento (refit=True)
hrs_result = hrs.fit(X_train_scaled, y_train_scaled, groups=groups)

Iniciando Halving Random Search (Testando as melhores combinações eficientemente)...
n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 10
min_resources_: 10
max_resources_: 8000
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 50
n_resources: 10
Fitting 5 folds for each of 50 candidates, totalling 250 fits


2025-10-15 13:50:24.252566: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 13:50:24.253168: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 13:50:24.311704: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-15 13:50:24.330825: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 13:50:24.331470: I tensorflow/core/util/port.cc:153] oneDNN custom 

[CV] END batch_size=64, epochs=5, model__dropout_rate=0.1733739159464655, model__learning_rate=0.007818940902700416, model__neurons=52; total time=   1.8s


/home/renato/.cache/pypoetry/virtualenvs/capacitacao-DnesvXxz-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END batch_size=64, epochs=5, model__dropout_rate=0.1733739159464655, model__learning_rate=0.007818940902700416, model__neurons=52; total time=   1.8s
[CV] END batch_size=64, epochs=5, model__dropout_rate=0.1733739159464655, model__learning_rate=0.007818940902700416, model__neurons=52; total time=   1.7s
[CV] END batch_size=64, epochs=5, model__dropout_rate=0.1733739159464655, model__learning_rate=0.007818940902700416, model__neurons=52; total time=   1.8s


...

[CV] END batch_size=64, epochs=20, model__dropout_rate=0.4046478461314871, model__learning_rate=0.0024526116855247567, model__neurons=69; total time=   2.5s
[CV] END batch_size=64, epochs=20, model__dropout_rate=0.4046478461314871, model__learning_rate=0.0024526116855247567, model__neurons=69; total time=   2.7s
[CV] END batch_size=32, epochs=20, model__dropout_rate=0.32449737033908044, model__learning_rate=0.0038909760600625196, model__neurons=206; total time=   3.1s


/home/renato/.cache/pypoetry/virtualenvs/capacitacao-DnesvXxz-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-15 13:52:00.006409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 13:52:00.006965: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 13:52:00.503530: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 13:52:00.5

[CV] END batch_size=32, epochs=10, model__dropout_rate=0.31573689676626027, model__learning_rate=0.008093657536124219, model__neurons=123; total time=   1.8s
[CV] END batch_size=64, epochs=20, model__dropout_rate=0.4046478461314871, model__learning_rate=0.0024526116855247567, model__neurons=69; total time=   2.9s
[CV] END batch_size=32, epochs=10, model__dropout_rate=0.31573689676626027, model__learning_rate=0.008093657536124219, model__neurons=123; total time=   1.9s


/home/renato/.cache/pypoetry/virtualenvs/capacitacao-DnesvXxz-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/renato/.cache/pypoetry/virtualenvs/capacitacao-DnesvXxz-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END batch_size=32, epochs=10, model__dropout_rate=0.31573689676626027, model__learning_rate=0.008093657536124219, model__neurons=123; total time=   2.0s


In [ ]:
print("\n" + "="*50)
print("RESULTADOS FINAIS DO HALVING RANDOM SEARCH")
print("="*50)

# Melhor pontuação
print(f"Melhor R² Médio: {hrs_result.best_score_:.4f}")

# Melhor combinação de hiperparâmetros
print("Melhores Parâmetros:")
print(hrs_result.best_params_)

# Obter o melhor modelo treinado
best_nn_model = hrs_result.best_estimator_

# --- AVALIAÇÃO FINAL ---


# O reshape é necessário pois o KerasRegressor.predict() retorna 1D por padrão no Scikit-Learn Wrapper
y_pred_scaled = best_nn_model.predict(X_train_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)

# 4. Prepara o target verdadeiro para comparação. Garante que y_train seja convertido de um pandas Series para um Numpy array, que é o objeto que sai da inverse_transform acima
y_true = y_train.values.reshape(-1, 1)


# Cálculo das métricas no conjunto de treino
final_mse = mean_squared_error(y_true, y_pred_original) 
final_rmse = root_mean_squared_error(y_true, y_pred_original) 
final_r2 = r2_score(y_true, y_pred_original)
final_mae = mean_absolute_error(y_true, y_pred_original) 


print("\nMétricas do Melhor Modelo (Avaliadas nos dados completos de Treino):")
print(f"  MSE (Erro Quadrático Médio): {final_mse:.8f} (Penaliza erros grandes)")
print(f"  RMSE (Erro Quadrático Médio): {final_rmse:.8f} (Penaliza erros grandes)")
print(f"  MAE (Erro Absoluto Médio):   {final_mae:.8f} (Unidade de 'r')")
print(f"  R2 (Ajuste):                 {final_r2:.4f} (Qualidade do ajuste)")
print("="*50)



## Observação

- Comparado com os resultados nos dados de treino do notebook 2, todas as métricas melhoraram.

# Visualizando os resultados

Abaixo, são mostradas duas visualizações:

- A distribuição dos valores preditos comparados com os reais,

- A distribuição dos resíduos.

In [ ]:
y_train.size

In [ ]:
y_train

In [ ]:
y_pred_original.size

In [ ]:
y_pred_original

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Suponha que você já tem:
# y_true_original: valores verdadeiros de 'r' (na escala original)
# y_pred_original: valores previstos de 'r' (na escala original)

# Crie a linha de identidade X=Y
min_val = min(y_train.min(), y_pred_original.min())
max_val = max(y_train.max(), y_pred_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_train, y_pred_original, alpha=0.6, s=20, label='Previsões')

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(ideal_line, ideal_line, color='red', linestyle='--', linewidth=2, label='Ajuste Perfeito (Y=X)')

plt.title(f'Previsões vs. Valores Reais de r (R² Final: {final_r2:.4f})', fontsize=14)
plt.xlabel('Valores Reais de r', fontsize=12)
plt.ylabel('Valores Previstos de r', fontsize=12)
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.gca().set_aspect('equal', adjustable='box') # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:
# Calcule os resíduos (True - Predicted)
residuals = y_true - y_pred_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor='black', alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Erro Zero')

plt.title('Distribuição dos Resíduos (Erros)', fontsize=14)
plt.xlabel('Resíduo (Real r - Previsto r)', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.show()

In [ ]:
# --- PREPARAÇÃO ---
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_true_flat = y_true.ravel()
y_pred_flat = y_pred_original.ravel()

df_results = pd.DataFrame({
    'y_true': y_true_flat,
    'y_pred': y_pred_flat
})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = df_results.groupby('y_true')['y_pred'].agg(['mean', 'std']).reset_index()

# Renomear as colunas para clareza
df_grouped.columns = ['y_true', 'y_pred_mean', 'y_pred_std']


# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped['y_pred_std_safe'] = df_grouped['y_pred_std'].replace(0, 1e-6)
df_grouped['Standardized_Residual'] = (
    np.abs(df_grouped['y_true'] - df_grouped['y_pred_mean']) / df_grouped['y_pred_std_safe']
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

## Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

Uma forma mais útil de visualizar os dados é portanto colapsar os 10 dados referentes à cada valor do target em seu valor médio e desvio padrão.

Além disso, plota-se gráfico de resíduos padronizados, que ajuda  visualizar o número de observações que se distanciam do valor real em mais de um desvio-padrão.



In [ ]:
# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2, 1, 
    figsize=(8, 10), 
    sharex=True, # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={'hspace': 0.05}, # Reduz o espaço entre os subplots
    constrained_layout=True 
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped['y_true'],             
    y=df_grouped['y_pred_mean'],            
    yerr=df_grouped['y_pred_std'],          
    fmt='o',                                
    capsize=4,                              
    alpha=0.7,
    label='Previsão Média ± 1 DP'
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(ideal_line, ideal_line, color='red', linestyle='--', linewidth=2, label='Ajuste Perfeito (Y=X)')

ax1.set_title(f'Análise de Previsão Agregada (R² Final: {final_r2:.4f})', fontsize=14)
ax1.set_ylabel('Valores Previstos Médios de r', fontsize=12)
ax1.grid(True, linestyle=':', alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped['y_true'], 
    df_grouped['Standardized_Residual'], 
    alpha=0.7, 
    s=30, 
    label='|Resíduo| / DP'
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, 
    color='red', 
    linestyle='-', 
    linewidth=2, 
    label='Limite de 1 Desvio Padrão'
)
ax2.axhline(y=2.0, color='orange', linestyle='--', linewidth=1, label='Limite de 2 DP')


ax2.set_xlabel('Valores Reais de r (Y_true)', fontsize=12)
ax2.set_ylabel('Resíduo Padronizado (|Y - Ŷ| / DP)', fontsize=12)
ax2.grid(True, linestyle=':', alpha=0.7)
ax2.legend()

plt.show()

## Obsevação

- Houve uma melhora nas métricas, mas fica evidente nas visualizações acima que houve uma piora nas previsões para valores muito baixo do target 'r'.

- O modelo tende a errar mais para valores de r próximos de 0. 

# Aplicando o modelo no conjunto de teste

In [ ]:
teste = pd.read_csv('../data/02_intermediate/teste_b_df.csv')

In [ ]:
teste

In [ ]:
X_test = teste.drop(columns=['r'])
y_test = teste['r']

## Criando features extras

Todas as features criadas para treinar o modelo devem também ser criadas ao se aplicar os dados de teste. 

In [ ]:
X_test = cria_feature_media_ponderada(X_test)

In [ ]:
X_test

## Selecionando features

In [ ]:
X_test = X_test[selected_feature_names]

# Normalizando os dados de teste

In [ ]:
X_test_scaled = X_scaler.fit_transform(X_test)

In [ ]:
X_test_scaled

In [ ]:
X_test_scaled.max()

In [ ]:
X_test_scaled.min()

In [ ]:
# Escalonamento do Target (Y)
# Reformatar y_train para que o scaler funcione (de Series para 2D array/DataFrame)
y_test_2d = y_test.values.reshape(-1, 1)

# Obtem o melhor modelo do Halving Random Search
best_nn_model = hrs_result.best_estimator_

# A previsão é feita na escala Z-Score (escalada)
y_pred_test_scaled = best_nn_model.predict(X_test_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_test_original = y_scaler.inverse_transform(y_pred_test_scaled)


print("Previsão no Conjunto de Teste concluída com sucesso.")

In [ ]:
X_test_scaled.max()

In [ ]:
y_pred_test_scaled.max()

In [ ]:
X_test_scaled.min()

In [ ]:
y_pred_test_scaled.min()

In [ ]:
y_pred_test_original.max()

In [ ]:
y_pred_test_original.min()

In [ ]:
# Cálculo das 3 Métricas de Avaliação Finais:
mse_test = mean_squared_error(y_test_2d, y_pred_test_original)
rmse_test = root_mean_squared_error(y_test_2d, y_pred_test_original)
mae_test = mean_absolute_error(y_test_2d, y_pred_test_original)
r2_test = r2_score(y_test_2d, y_pred_test_original)

print("\n" + "="*50)
print("AVALIAÇÃO DE DESEMPENHO NO CONJUNTO DE TESTE")
print("="*50)
print(f"MSE (Erro Quadrático Médio): {mse_test:.8f}")
print(f"RMSE (Erro Absoluto Médio):   {rmse_test:.8f} (Erro médio na unidade de 'r')")
print(f"MAE (Erro Absoluto Médio):   {mae_test:.8f} (Erro médio na unidade de 'r')")
print(f"R2 (Ajuste):                 {r2_test:.4f}")
print("="*50)

In [ ]:
# Crie a linha de identidade X=Y
min_val = min(y_test_2d.min(), y_pred_test_original.min())
max_val = max(y_test_2d.max(), y_pred_test_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_test_2d, y_pred_test_original, alpha=0.6, s=20, label='Previsões')

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(ideal_line, ideal_line, color='red', linestyle='--', linewidth=2, label='Ajuste Perfeito (Y=X)')

plt.title(f'Previsões vs. Valores Reais de r (R² Final: {r2_test:.4f})', fontsize=14)
plt.xlabel('Valores Reais de r', fontsize=12)
plt.ylabel('Valores Previstos de r', fontsize=12)
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.gca().set_aspect('equal', adjustable='box') # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:


# Calcule os resíduos (True - Predicted)
residuals = y_test_2d - y_pred_test_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor='black', alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Erro Zero')

plt.title('Distribuição dos Resíduos (Erros)', fontsize=14)
plt.xlabel('Resíduo (Real r - Previsto r)', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.show()

## Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

A visualização com barras de erros também é feita abaixo utilizando-se os dados de teste.

In [ ]:
# --- PREPARAÇÃO ---
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_test_flat = y_test_2d.ravel()
y_pred_test_flat = y_pred_test_original.ravel()

df_results = pd.DataFrame({
    'y_test': y_test_flat,
    'y_test_pred': y_pred_test_flat
})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = df_results.groupby('y_test')['y_test_pred'].agg(['mean', 'std']).reset_index()

# Renomear as colunas para clareza
df_grouped.columns = ['y_test', 'y_test_pred_mean', 'y_test_pred_std']

# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped['y_test_pred_std_safe'] = df_grouped['y_test_pred_std'].replace(0, 1e-6)
df_grouped['Standardized_Residual'] = (
    np.abs(df_grouped['y_test'] - df_grouped['y_test_pred_mean']) / df_grouped['y_test_pred_std_safe']
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

In [ ]:
# --- VISUALIZAÇÃO COM GRÁFICO DE DUPLO EIXO ---

# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2, 1, 
    figsize=(8, 10), 
    sharex=True, # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={'hspace': 0.05}, # Reduz o espaço entre os subplots
    constrained_layout=True 
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped['y_test'],             
    y=df_grouped['y_test_pred_mean'],            
    yerr=df_grouped['y_test_pred_std'],          
    fmt='o',                                
    capsize=4,                              
    alpha=0.7,
    label='Previsão Média ± 1 DP'
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(ideal_line, ideal_line, color='red', linestyle='--', linewidth=2, label='Ajuste Perfeito (Y=X)')

ax1.set_title(f'Análise de Previsão Agregada (R² Final: {final_r2:.4f})', fontsize=14)
ax1.set_ylabel('Valores Previstos Médios de r', fontsize=12)
ax1.grid(True, linestyle=':', alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped['y_test'], 
    df_grouped['Standardized_Residual'], 
    alpha=0.7, 
    s=30, 
    label='|Resíduo| / DP'
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, 
    color='red', 
    linestyle='-', 
    linewidth=2, 
    label='Limite de 1 Desvio Padrão'
)
ax2.axhline(y=2.0, color='orange', linestyle='--', linewidth=1, label='Limite de 2 DP')


ax2.set_xlabel('Valores Reais de r(Y_true)', fontsize=12)
ax2.set_ylabel('Resíduo Padronizado (|Y - Ŷ| / DP)', fontsize=12)
ax2.grid(True, linestyle=':', alpha=0.7)
ax2.legend()

plt.show()

# Observação

- Ao utilizarmos apenas as 150 features mais importantes de acordo com o teste estatístico feito com o método da classe SelectKBest do scikit-learn, houve uma piora nas métricas do modelo. Principalmente na região central, onde  0.01 < r < 0.03.

# Sugestões para trabalhos futuros

- Fazer uma seleção de features utilizando-se um modelo linear, como, por exemplo, uma regressão linear.

- Testar o resultado do modelo eliminando menos features.

- Criar novas features com os dados de entrada.

- Aumentar a rede de parâmetros da validação cruzada.